In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('popular')
nltk.download('stopwords') #stopwords
nltk.download("punkt") #ponctuation
from nltk.tokenize import word_tokenize
# import re #si utilise regex
import spacy #lemmatizer
# !spacy download fr_core_news_sm => Pour google collab
from nltk.stem import SnowballStemmer #stemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\cathe\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\cathe\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\cathe\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\cathe\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\cathe\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [2]:
'''
from sklearn.linear_model import LogisticRegression
'''



'\nfrom sklearn.linear_model import LogisticRegression\n'

<h4> 1 - Importe ce dataset de tweets dans un DataFrame.

In [3]:
link = "https://github.com/murpi/wilddata/raw/master/quests/tweets.zip"

df= pd.read_csv(link)

In [4]:
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
df.shape

(27481, 4)

<h4> 2 - Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). <br />
 Quel est le pourcentage de tweets positifs/négatifs ?

In [6]:
#count values in df|'sentiment']
df['sentiment'].value_counts()

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [7]:
# drop neutral snetiment in df

df.drop(df[df['sentiment']== 'neutral'].index, inplace = True)
df.reset_index(inplace = True, drop = True)
df


,textID,text,selected_text,sentiment
0,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
1,088c60f138,my boss is bullying me...,bullying me,negative
2,9642c003ef,what interview! leave me alone,leave me alone,negative
3,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
4,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive
...,...,...,...,...
16358,b78ec00df5,enjoy ur night,enjoy,positive
16359,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
16360,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
16361,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive


In [8]:
# calcul percent tweet negative and positive

percent = df['sentiment'].value_counts("positive")*100
percent


positive    52.447595
negative    47.552405
Name: sentiment, dtype: float64

In [9]:
print(f"The percent of positive sentiment is {round(percent[0],2)} %")
print(f"The percent of negative sentiment is {round(percent[1],2)} %")

The percent of positive sentiment is 52.45 %
The percent of negative sentiment is 47.55 %


<h4> 3 - Crée une fonction que tu appelleras clean qui prend pour paramètre une phrase (donc un texte str) et qui retourne un texte (str) de tokens après application d'un stemmer ou d'un lemmatizer, séparés par des espaces.

In [10]:
texte = 'Le cinéma et l’audiovisuel sont bouleversés par l’expansion des plates-formes Internet'
type(texte)

str

In [11]:
# retrait ponctuation avec regex
# texte_clean = re.sub(r'[^\w\s]',"",texte)
# texte_clean

In [12]:
txt_tokens = nltk.word_tokenize(texte.lower())
txt_tokens

['le',
 'cinéma',
 'et',
 'l',
 '’',
 'audiovisuel',
 'sont',
 'bouleversés',
 'par',
 'l',
 '’',
 'expansion',
 'des',
 'plates-formes',
 'internet']

In [13]:
# retrait ponctuation
text_without_punct= [word for word in txt_tokens if word.isalnum()]
text_without_punct

['le',
 'cinéma',
 'et',
 'l',
 'audiovisuel',
 'sont',
 'bouleversés',
 'par',
 'l',
 'expansion',
 'des',
 'internet']

In [14]:
# retrait stopword 
text_without_sw = [word for word in text_without_punct if not word in stopwords.words()]
text_without_sw


['cinéma', 'audiovisuel', 'bouleversés', 'expansion', 'internet']

In [15]:
# Utilisation d'un stemmer

stem_fr = SnowballStemmer('french')

for word in text_without_sw:
  print(stem_fr.stem(word))

cinem
audiovisuel
boulevers
expans
internet


In [16]:
# Utilisation d'un lemmatizer

# Import des règles françaises
nlp = spacy.load('fr_core_news_sm')


# spacy découpe automatiquement en tokens avec cette syntaxe :
sent_tokens = nlp(str(text_without_sw))

for token in sent_tokens:
    print(token, token.lemma_)



[ [
' '
cinéma cinéma
' '
, ,
' '
audiovisuel audiovisuel
' '
, ,
' '
bouleversés bouleverser
' '
, ,
' '
expansion expansion
' '
, ,
' '
internet internet
' '
] ]


<h4> 4- Récupère la liste des stopwords anglais depuis NLTK, et copie-la dans une liste stopwordsenglish. Complète ta fonction clean pour qu'elle supprime la ponctuation et les stopwords.

Déjà fait ci-dessus avec : <br/>
nltk.download('stopwords') ==> Pour les stopwords <br/>
nltk.download("punkt") ==> Pour la ponctuation mais... <br/>

je refais pour coller à la consigne !

In [17]:
stopwordsenglish = set(stopwords.words('english'))


def clean (item) :
    wordsFiltered = []
    for i in range(len(item)):  
        words = word_tokenize(item)
        print(words)


        for w in words:
            if w not in stopwordsenglish:
                wordsFiltered.append(w)
        return wordsFiltered

# pour tester ma fonction
data = df['text'][0]
clean(data)

['Sooo', 'SAD', 'I', 'will', 'miss', 'you', 'here', 'in', 'San', 'Diego', '!', '!', '!']


['Sooo', 'SAD', 'I', 'miss', 'San', 'Diego', '!', '!', '!']

<h4> 5 - Applique cette fonction clean à la colonne text de ton DataFrame. <br/>
Stocke le résultat dans une nouvelle colonne clean du DataFrame. (Le traitement peut durer 2 ou 3 minutes)

In [18]:
df['text_clean'] = df['text'].apply(clean)

['Sooo', 'SAD', 'I', 'will', 'miss', 'you', 'here', 'in', 'San', 'Diego', '!', '!', '!']
['my', 'boss', 'is', 'bullying', 'me', '...']
['what', 'interview', '!', 'leave', 'me', 'alone']
['Sons', 'of', '*', '*', '*', '*', ',', 'why', 'couldn', '`', 't', 'they', 'put', 'them', 'on', 'the', 'releases', 'we', 'already', 'bought']
['2am', 'feedings', 'for', 'the', 'baby', 'are', 'fun', 'when', 'he', 'is', 'all', 'smiles', 'and', 'coos']
['Journey', '!', '?', 'Wow', '...', 'u', 'just', 'became', 'cooler', '.', 'hehe', '...', '(', 'is', 'that', 'possible', '!', '?', ')']
['I', 'really', 'really', 'like', 'the', 'song', 'Love', 'Story', 'by', 'Taylor', 'Swift']
['My', 'Sharpie', 'is', 'running', 'DANGERously', 'low', 'on', 'ink']
['i', 'want', 'to', 'go', 'to', 'music', 'tonight', 'but', 'i', 'lost', 'my', 'voice', '.']
['Uh', 'oh', ',', 'I', 'am', 'sunburned']
['S', '`', 'ok', ',', 'trying', 'to', 'plot', 'alternatives', 'as', 'we', 'speak', '*', 'sigh', '*']
['i', '`', 've', 'been', 'sick', 

In [19]:
df.head()

,textID,text,selected_text,sentiment,text_clean
0,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,"[Sooo, SAD, I, miss, San, Diego, !, !, !]"
1,088c60f138,my boss is bullying me...,bullying me,negative,"[boss, bullying, ...]"
2,9642c003ef,what interview! leave me alone,leave me alone,negative,"[interview, !, leave, alone]"
3,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,"[Sons, *, *, *, *, ,, `, put, releases, alread..."
4,6e0c6d75b1,2am feedings for the baby are fun when he is a...,fun,positive,"[2am, feedings, baby, fun, smiles, coos]"


In [20]:
type(df['text_clean'])

pandas.core.series.Series

In [21]:
type(str(df['text']))

str

<h4> 6 - Copie la colonne text_clean dans une Série X, et la colonne sentiment dans une Série y. <br />
Applique un train-test split avec la taille du jeu d'entrainement à 0.75 avec le random_state = 32.

In [22]:
# # The TFIDF Vectorizer should expect an array of strings so add this code .apply(lambda x: ' '.join(x)) after X and it should work.
X = df['text_clean'].apply(lambda x: ' '.join(x)) 
y = df['sentiment']

In [29]:
df['sentiment'].shape

(16363,)

In [28]:
df['text_clean'].shape

(16363,)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=32, train_size = 0.75)
print("The length of the initial dataset is :", len(X))
print("The length of the train dataset is   :", len(X_train))
print("The length of the test dataset is    :", len(X_test))

The length of the initial dataset is : 16363
The length of the train dataset is   : 12272
The length of the test dataset is    : 4091


<h4> 7 - Applique un CountVectorizer et entraine des modèles de classification.



In [24]:
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

In [25]:
X_train_CV = vectorizer.transform(X_train)
X_train_CV

<12272x15779 sparse matrix of type '<class 'numpy.int64'>'
	with 93995 stored elements in Compressed Sparse Row format>

In [26]:
X_test_CV = vectorizer.transform(X_test)
X_test_CV

<4091x15779 sparse matrix of type '<class 'numpy.int64'>'
	with 28161 stored elements in Compressed Sparse Row format>

In [27]:
newmodel = LogisticRegression(max_iter=150).fit(X_train_CV, y_train)

#Print the accuracy score on the train set and on the test set
print(f"The accuracy score for training set: {round(newmodel.score(X_train_CV, y_train),3)}")
print(f"The accuracy score  for test set: {round(newmodel.score(X_test_CV, y_test),3)}")

NameError: name 'LogisticRegression' is not defined

<h4> 4 - Applique un TfidfVectorizer  et entraine des modèles de classification.

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

TfidfVectorizer()

In [ ]:
X_train_CV = vectorizer.transform(X_train)
X_train_CV

<12272x15779 sparse matrix of type '<class 'numpy.float64'>'
	with 93995 stored elements in Compressed Sparse Row format>

In [ ]:
X_test_CV = vectorizer.transform(X_test)
X_test_CV

<4091x15779 sparse matrix of type '<class 'numpy.float64'>'
	with 28161 stored elements in Compressed Sparse Row format>

In [ ]:
newmodel = LogisticRegression(max_iter=150).fit(X_train_CV, y_train)

#Print the accuracy score on the train set and on the test set
print(f"The accuracy score for training set: {round(newmodel.score(X_train_CV, y_train),3)}")
print(f"The accuracy score  for test set: {round(newmodel.score(X_test_CV, y_test),3)}")

The accuracy score for training set: 0.933
The accuracy score  for test set: 0.867


<h4> 9 - Compare les scores, quels paramètres permettent d'avoir les meilleurs scores ?

C'est le TfidfVectorizer qui permet d'avoir le meilleur score

<h4> 10 - BONUS : maintenant c'est à toi d'améliorer ton modèle : <br/>
En cherchant des paramètres de modèles : par gridsearch et crossvalidation par exemple ;<br/>
En changeant la préparation du texte : <br/>
Par exemple certaines ponctuations peuvent aider le modèle, le point d'exclamation notamment.